In [1]:
from pyspark import SparkConf, SparkContext
conf = SparkConf().setMaster("local").setAppName("spark_sql_basic2")
sc   = SparkContext(conf=conf)

# RDD만을 이용한 데이터 추출

In [3]:
movies_rdd = sc.parallelize([
    (1, ("어벤져스", "마블")),
    (2, ("슈퍼맨", "DC")),
    (3, ("배트맨", "DC")),
    (4, ("겨울왕국", "디즈니")),
    (5, ("아이언맨", "마블"))
])


attendances_rdd = sc.parallelize([
    (1, (13934592, "KR")),
    (2, (2182227,"KR")),
    (3, (4226242, "KR")),
    (4, (10303058, "KR")),
    (5, (4300365, "KR"))
])

In [ ]:
# 마블 영화 중 관객 수가 500만 이상인 영화를 가져오기

In [7]:
# CASE1. join 먼저, filter 나중에
movie_att = movies_rdd.join(attendances_rdd)
movie_att.take(5)

[(2, (('슈퍼맨', 'DC'), (2182227, 'KR'))),
 (4, (('겨울왕국', '디즈니'), (10303058, 'KR'))),
 (1, (('어벤져스', '마블'), (13934592, 'KR'))),
 (3, (('배트맨', 'DC'), (4226242, 'KR'))),
 (5, (('아이언맨', '마블'), (4300365, 'KR')))]

In [8]:
movie_att.filter(
    lambda x : x[1][0][1] == "마블" and x[1][1][0] > 5000000
).collect()

[(1, (('어벤져스', '마블'), (13934592, 'KR')))]

In [9]:
# CASE 2. filter 먼저, join 나중에
filtered_movies = movies_rdd.filter(lambda x : x[1][1] == '마블')
filtered_att = attendances_rdd.filter(lambda x : x[1][0] > 5000000)

filtered_movies.join(filtered_att).collect()

[(1, (('어벤져스', '마블'), (13934592, 'KR')))]

In [11]:
sc.stop()

# Spark SQL 사용해 보기

In [49]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").appName("spark-sql").getOrCreate()

In [33]:
# 컬럼 추가
movies = [
    (1, "어벤져스", "마블", 2012, 4, 26),
    (2, "슈퍼맨", "DC", 2013, 6, 13),
    (3, "배트맨", "DC", 2008, 8, 6),
    (4, "겨울왕국", "디즈니", 2014, 1, 16),
    (5, "아이언맨", "마블", 2008, 4, 30)
]

In [34]:
#스키마를 알아야 한다.
movie_schema = ["id", "name", "company", "year", "month", "day"]

# 2. 데이터 프레임 만들기

In [58]:
df = spark.createDataFrame(data=movies, schema=movie_schema)

In [16]:
df.dtypes

[('id', 'bigint'),
 ('name', 'string'),
 ('company', 'string'),
 ('year', 'bigint'),
 ('month', 'bigint'),
 ('day', 'bigint')]

In [19]:
df.show(3)

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  1|어벤져스|   마블|2012|    4| 26|
|  2|  슈퍼맨|     DC|2013|    6| 13|
|  3|  배트맨|     DC|2008|    8|  6|
+---+--------+-------+----+-----+---+
only showing top 3 rows



In [20]:
df.select("name").show()

+--------+
|    name|
+--------+
|어벤져스|
|  슈퍼맨|
|  배트맨|
|겨울왕국|
|아이언맨|
+--------+



In [25]:
df.select('company').show(3)

+-------+
|company|
+-------+
|   마블|
|     DC|
|     DC|
+-------+
only showing top 3 rows



In [26]:
df.filter(df.month >= 5).show()

+---+------+-------+----+-----+---+
| id|  name|company|year|month|day|
+---+------+-------+----+-----+---+
|  2|슈퍼맨|     DC|2013|    6| 13|
|  3|배트맨|     DC|2008|    8|  6|
+---+------+-------+----+-----+---+



In [30]:
df.select('year','month','day').show()

+----+-----+---+
|year|month|day|
+----+-----+---+
|2012|    4| 26|
|2013|    6| 13|
|2008|    8|  6|
|2014|    1| 16|
|2008|    4| 30|
+----+-----+---+



In [31]:
df.filter(df.year >= 2013).show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  2|  슈퍼맨|     DC|2013|    6| 13|
|  4|겨울왕국| 디즈니|2014|    1| 16|
+---+--------+-------+----+-----+---+



In [36]:
df.filter(df.company == '마블').show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  1|어벤져스|   마블|2012|    4| 26|
|  5|아이언맨|   마블|2008|    4| 30|
+---+--------+-------+----+-----+---+



In [46]:
df[df['company'].isin(['마블', 'DC'])].show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  1|어벤져스|   마블|2012|    4| 26|
|  2|  슈퍼맨|     DC|2013|    6| 13|
|  3|  배트맨|     DC|2008|    8|  6|
|  5|아이언맨|   마블|2008|    4| 30|
+---+--------+-------+----+-----+---+



In [60]:
df.createOrReplaceTempView("movies")

In [61]:
# 영화 이름만 가져오기

query = """

SELECT name
  FROM movies

"""
spark.sql(query).show()

+--------+
|    name|
+--------+
|어벤져스|
|  슈퍼맨|
|  배트맨|
|겨울왕국|
|아이언맨|
+--------+



In [10]:
# 2010년 이후에 개봉한 영화를 조회

query = """
SELECT * FROM movies WHERE year>=2010; 
"""
spark.sql(query).show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  1|어벤져스|   마블|2012|    4| 26|
|  2|  슈퍼맨|     DC|2013|    6| 13|
|  4|겨울왕국| 디즈니|2014|    1| 16|
+---+--------+-------+----+-----+---+



In [12]:
# 2012년도 이전에 개봉한 영화의 이름과 회사를 출력
query = """
SELECT name, company, year FROM movies WHERE 2012 >= year;
"""
spark.sql(query).show()

+--------+-------+----+
|    name|company|year|
+--------+-------+----+
|어벤져스|   마블|2012|
|  배트맨|     DC|2008|
|아이언맨|   마블|2008|
+--------+-------+----+



In [16]:
# like 문자열 데이터에서 특정 단어나 문장을 포함한 데이터를 찾을 때
# % 기호를 사용해서 문장이 매칭되는지 확인 가능!
# 제목이 ~~맨으로 끝나는 데이터의 모든 정보를 조회
query = """
SELECT * FROM movies WHERE name LIKE "%맨";
"""
spark.sql(query).show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  2|  슈퍼맨|     DC|2013|    6| 13|
|  3|  배트맨|     DC|2008|    8|  6|
|  5|아이언맨|   마블|2008|    4| 30|
+---+--------+-------+----+-----+---+



In [18]:

# BETWEEN 특정 데이터와 데이터 사이를 조회

# 개봉 월이 4 ~ 8월 사이. 4 <= 개봉월 <= 8
query = """
SELECT * FROM movies WHERE month BETWEEN 4 AND 8 ;
"""
spark.sql(query).show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  1|어벤져스|   마블|2012|    4| 26|
|  2|  슈퍼맨|     DC|2013|    6| 13|
|  3|  배트맨|     DC|2008|    8|  6|
|  5|아이언맨|   마블|2008|    4| 30|
+---+--------+-------+----+-----+---+



In [46]:
spark.stop()

# Join 구현하기

In [50]:

attendances = [
    (1, 13934592., "KR"),
    (2, 2182227.,"KR"),
    (3, 4226242., "KR"),
    (4, 10303058., "KR"),
    (5, 4300365., "KR")
]

In [51]:
# 직접 스키마 지정해 보기
from pyspark.sql.types import StringType, FloatType\
    , IntegerType\
    , StructType, StructField

In [52]:
att_schema = StructType([ # 모든 컬럼의 타입을 통칭 - 컬럼 데이터의 집합
    StructField("id", IntegerType(), True), # StructField : 컬럼
    StructField("att", FloatType(), True),
    StructField("theater_country", StringType(), True)
])

In [53]:
att_df = spark.createDataFrame(
    data=attendances,
    schema=att_schema
)
att_df.dtypes

[('id', 'int'), ('att', 'float'), ('theater_country', 'string')]

In [54]:
att_df.createOrReplaceTempView("att")

In [62]:
# df 와 join
query = '''
SELECT movies.id, movies.name, movies.company, att.att 
FROM movies
JOIN att ON movies.id = att.id 
'''
spark.sql(query).show()

+---+--------+-------+-----------+
| id|    name|company|        att|
+---+--------+-------+-----------+
|  1|어벤져스|   마블|1.3934592E7|
|  2|  슈퍼맨|     DC|  2182227.0|
|  3|  배트맨|     DC|  4226242.0|
|  4|겨울왕국| 디즈니|1.0303058E7|
|  5|아이언맨|   마블|  4300365.0|
+---+--------+-------+-----------+



In [ ]:
# 데이터 프레임 API

In [ ]:
# select
df.select("*").collect()

In [ ]:
df.select("name", "company").collect()

In [ ]:
df.select(df.name, (df.year-2000).alias("year")).show()

In [ ]:
# agg : Aggreagte의 약자로써, 그룹핑 후 데이터를 하나로 합쳐주는 역할
df.agg({"id": "count"}).collect()

In [ ]:
from pyspark.sql import functions as F
df.agg(F.min(df.year)).collect()

In [ ]:
df.groupBy().avg().collect()

In [ ]:
# 회사별 개봉월의 평균
df.groupBy('company').agg({"month": "mean"}).collect()

In [ ]:
# 회사 별 월 별 영화 개수 정보


In [ ]:
# join : 다른 데이터 프레임과 사용자가 지정한 컬럼을 기준으로 합치는 작업
df.join(att_df, 'id').select(df.name, att_df.att).show()

In [ ]:
# select, where, orderBy 절 사용
marvel_df = df.select("name", "company", "year").where("company=='마블'").orderBy("id")
marvel_df.collect()

In [65]:
spark.stop()

In [ ]:
# SQL 최적화

In [100]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("trip_count_sql").getOrCreate()

In [79]:
import os
print(os.getcwd())

/home/jovyan/work/practice_virtual-environment


In [89]:
trip_file = "learning_spark_data/fhvhv_tripdata_2020-03.csv"

In [91]:
# inferSchema : 자동으로 스키마 예측하게 하기
data = spark.read.csv(trip_file, inferSchema=True, header=True)

In [92]:
data.createOrReplaceTempView("mobility_data")

In [93]:
query = """
select *
from mobility_data
limit 5
"""
spark.sql(query).show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0005|              B02510|2020-03-01 00:03:40|2020-03-01 00:23:39|          81|         159|   NULL|
|           HV0005|              B02510|2020-03-01 00:28:05|2020-03-01 00:38:57|         168|         119|   NULL|
|           HV0003|              B02764|2020-03-01 00:03:07|2020-03-01 00:15:04|         137|         209|      1|
|           HV0003|              B02764|2020-03-01 00:18:42|2020-03-01 00:38:42|         209|          80|   NULL|
|           HV0003|              B02764|2020-03-01 00:44:24|2020-03-01 00:58:44|         256|         226|   NULL|
+-----------------+--------------------+-------------------+-------------------+

In [ ]:
# 스파크 SQL을 사용하는 이유

In [94]:
query = """

select split(pickup_datetime, ' ')[0] as pickup_date, count(*) as trips
from mobility_data

group by pickup_date
"""

spark.sql(query).show()

+-----------+------+
|pickup_date| trips|
+-----------+------+
| 2020-03-03|697880|
| 2020-03-02|648986|
| 2020-03-01|784246|
| 2020-03-06|872012|
| 2020-03-05|731165|
| 2020-03-04|707879|
| 2020-03-09|628940|
| 2020-03-08|731222|
| 2020-03-07|886071|
| 2020-03-10|626474|
| 2020-03-12|643257|
| 2020-03-11|628601|
| 2020-03-16|391518|
| 2020-03-13|660914|
| 2020-03-15|448125|
| 2020-03-14|569397|
| 2020-03-26|141607|
| 2020-03-25|141088|
| 2020-03-20|261900|
| 2020-03-24|141686|
+-----------+------+
only showing top 20 rows



In [95]:
# 실행 계획 살펴보기
spark.sql(query).explain(True)

== Parsed Logical Plan ==
'Aggregate ['pickup_date], ['split('pickup_datetime,  )[0] AS pickup_date#324, 'count(1) AS trips#325]
+- 'UnresolvedRelation [mobility_data], [], false

== Analyzed Logical Plan ==
pickup_date: string, trips: bigint
Aggregate [split(cast(pickup_datetime#251 as string),  , -1)[0]], [split(cast(pickup_datetime#251 as string),  , -1)[0] AS pickup_date#324, count(1) AS trips#325L]
+- SubqueryAlias mobility_data
   +- View (`mobility_data`, [hvfhs_license_num#249,dispatching_base_num#250,pickup_datetime#251,dropoff_datetime#252,PULocationID#253,DOLocationID#254,SR_Flag#255])
      +- Relation [hvfhs_license_num#249,dispatching_base_num#250,pickup_datetime#251,dropoff_datetime#252,PULocationID#253,DOLocationID#254,SR_Flag#255] csv

== Optimized Logical Plan ==
Aggregate [_groupingexpression#329], [_groupingexpression#329 AS pickup_date#324, count(1) AS trips#325L]
+- Project [split(cast(pickup_datetime#251 as string),  , -1)[0] AS _groupingexpression#329]
   +- Rel

In [96]:
# 두번째 쿼리
spark.sql("""select 
                pickup_date, 
                count(*) as trips
             from ( select
                          split(pickup_datetime, ' ')[0] as pickup_date
                          from mobility_data )
             group by pickup_date""").explain(True)

== Parsed Logical Plan ==
'Aggregate ['pickup_date], ['pickup_date, 'count(1) AS trips#333]
+- 'SubqueryAlias __auto_generated_subquery_name
   +- 'Project ['split('pickup_datetime,  )[0] AS pickup_date#332]
      +- 'UnresolvedRelation [mobility_data], [], false

== Analyzed Logical Plan ==
pickup_date: string, trips: bigint
Aggregate [pickup_date#332], [pickup_date#332, count(1) AS trips#333L]
+- SubqueryAlias __auto_generated_subquery_name
   +- Project [split(cast(pickup_datetime#251 as string),  , -1)[0] AS pickup_date#332]
      +- SubqueryAlias mobility_data
         +- View (`mobility_data`, [hvfhs_license_num#249,dispatching_base_num#250,pickup_datetime#251,dropoff_datetime#252,PULocationID#253,DOLocationID#254,SR_Flag#255])
            +- Relation [hvfhs_license_num#249,dispatching_base_num#250,pickup_datetime#251,dropoff_datetime#252,PULocationID#253,DOLocationID#254,SR_Flag#255] csv

== Optimized Logical Plan ==
Aggregate [pickup_date#332], [pickup_date#332, count(1) AS tri

In [97]:
spark.stop()

In [99]:
# 운행, zone 데이터 조인

In [102]:
trip_file = "learning_spark_data/fhvhv_tripdata_2020-03.csv"
zone_file = "learning_spark_data/taxi+_zone_lookup.csv"

In [103]:
trip_df = spark.read.format('csv')\
        .option('header','true')\
        .option('inferSchema','true')\
        .load(trip_file)
zone_df = spark.read.format('csv')\
        .option('header','true')\
        .option('inferSchema','true')\
        .load(zone_file)

In [104]:
## 승차 Location(PULocationID)별 개수 세기
# 하차 Location(DOLocationID)별 개수 세기
#HV0003 운송사업자의 승차 지역별 트립 건수를 집계하고, 
#가장 많은 운송사업자순으로 정렬하는 분석 쿼리  hvfhs_license_num
#운송사별 운행 건수 비교
#승차 위치 Borough별 운행 건수
#서비스 존별 승차/하차 건수

In [105]:
trip_df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: integer (nullable = true)



In [106]:
zone_df.printSchema()

root
 |-- LocationID: integer (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)



In [121]:
trip_df_pd = trip_df.limit(50).toPandas()
trip_df_pd.head(3)

,hvfhs_license_num,dispatching_base_num,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,SR_Flag
0,HV0005,B02510,2020-03-01 00:03:40,2020-03-01 00:23:39,81,159,NaN
1,HV0005,B02510,2020-03-01 00:28:05,2020-03-01 00:38:57,168,119,NaN
2,HV0003,B02764,2020-03-01 00:03:07,2020-03-01 00:15:04,137,209,1.0


In [119]:
## 승차 Location(PULocationID)별 개수 세기
from pyspark.sql.functions import countDistinct
trip_df.select(countDistinct('PULocationID')).show()

+----------------------------+
|count(DISTINCT PULocationID)|
+----------------------------+
|                         262|
+----------------------------+



In [122]:
trip_df.show(truncate=False)

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|pickup_datetime    |dropoff_datetime   |PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|HV0005           |B02510              |2020-03-01 00:03:40|2020-03-01 00:23:39|81          |159         |NULL   |
|HV0005           |B02510              |2020-03-01 00:28:05|2020-03-01 00:38:57|168         |119         |NULL   |
|HV0003           |B02764              |2020-03-01 00:03:07|2020-03-01 00:15:04|137         |209         |1      |
|HV0003           |B02764              |2020-03-01 00:18:42|2020-03-01 00:38:42|209         |80          |NULL   |
|HV0003           |B02764              |2020-03-01 00:44:24|2020-03-01 00:58:44|256         |226         |NULL   |
|HV0003           |B02682              |2020-03-01 00:17:23|2020-03-01 00:39:35|

In [118]:
# 하차 Location(DOLocationID)별 개수 세기
from pyspark.sql.functions import countDistinct
trip_df.select(countDistinct('DOLocationID')).show()

+----------------------------+
|count(DISTINCT DOLocationID)|
+----------------------------+
|                         262|
+----------------------------+



In [133]:
# HV0003 운송사업자의 승차 지역별 트립 건수를 집계,
#가장 많은 운송사업자순으로 정렬하는 분석 쿼리  hvfhs_license_num
from pyspark.sql.functions import col, count, desc
HV003_df = trip_df.filter(col('hvfhs_license_num') == 'HV0003')
HV003_df.groupBy('PULocationID').agg(count('*').alias('tripcount'))\
        .orderBy(desc('tripcount')).show()

+------------+---------+
|PULocationID|tripcount|
+------------+---------+
|          61|   163091|
|          76|   134198|
|         132|   114179|
|          79|   112017|
|          37|   110150|
|          42|   108070|
|         138|   104119|
|         244|    97324|
|          89|    95724|
|          39|    94484|
|         231|    94155|
|           7|    92676|
|          17|    90352|
|         161|    90261|
|         225|    88749|
|         234|    88372|
|         230|    86870|
|         188|    84347|
|          35|    82764|
|         168|    82396|
+------------+---------+
only showing top 20 rows



In [136]:
#운송사별 운행 건수 비교
from pyspark.sql.functions import concat
trip_df.withColumn("PU_DO", concat(col("PULocationID"), col("DOLocationID")))\
        .groupBy('PULocationID')\
        .agg(count('PU_DO').alias('totaltripcount'))\
        .orderBy(col('totaltripcount').desc())\
        .show()

+------------+--------------+
|PULocationID|totaltripcount|
+------------+--------------+
|          61|        222094|
|          79|        183821|
|          76|        168311|
|         132|        163734|
|         138|        155876|
|          37|        155388|
|          42|        143389|
|         231|        135712|
|         234|        132693|
|         161|        128751|
|         244|        126621|
|          17|        126228|
|           7|        125458|
|          89|        124289|
|         225|        124001|
|         170|        122731|
|         230|        121628|
|         181|        121576|
|          48|        120732|
|          39|        116669|
+------------+--------------+
only showing top 20 rows



In [ ]:
#승차 위치 Borough별 운행 건수

In [ ]:
#서비스 존별 승차/하차 건수

In [137]:
spark.stop()